<center><h1><strong> Informe Tarea 1 - Beer Clustering </strong></h1></center>

<center><h3> Matías Pacheco &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;201204619-k</h3></center>
<center><h3> Juan Pablo Castillo Vera &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 201204595-9</h3></center>

## Descripción del dataset

El dataset Beer Reviews es una colección de 1.590.892 críticas de cervezas que se encuentran disponibles en el sitio http://beeradvocate.com/. Éstas críticas toman en consideración características como aroma, apariencia y sabor para asignarle un puntaje a la cerveza que está siendo evaluada.

Cada registro del dataset completo está compuesto por las siguientes columnas:

|Columna             |Significado            |Tipo       |
|---                 |---                    |---        |
|brewery_id          |ID cervecería          |Numérico   |
|brewery_name        |Nombre Cervecería      |Texto      |
|review_time         |Tiempo de evaluación   |Numérico   |
|review_overall      |Puntaje del review     |Numérico   |
|review_aroma        |Puntaje aroma          |Numérico   |
|review_appearence   |Puntaje apariencia     |Numérico   |
|review_profilename  |Nombre del evaluador   |Texto      |
|beer_style          |Tipo de cerveza        |Texto      |
|review_palate       |Puntaje paladar        |Numérico   |
|review_taste        |Puntaje gusto          |Numérico   |
|beer_abv            |Grado alcohólico       |Numérico   |
|beer_beerid         |ID cerveza             |Numérico   |

Cabe señalar que como pre-procesamiento de los datos, se setearon en cero todos aquellos campos numéricos del dataset que estuvieran vacíos (string '').


## Análisis de Clustering

Se realizará un análisis de clustering con el dataset con cada uno de los siguientes cinco algoritmos:
* K-Means
* Minibatch K-Means
* HAC Complete
* Ward
* DBScan

### Observaciones y Supuestos

* Antes del procesamiento de los datos, éstos fueron normalizados debido a que antes de aplicar reduccion de dimensionalidad por PCA, los datos deben normalizarse para que todos se encuentren dentro de la misma escala y asi los valores considerados "outlier" no interfieren significativamente en el tratamiento de los datos al reducir su dimensionalidad.

In [ ]:
from sklearn.preprocessing import StandardScaler
...
reviews = StandardScaler().fit_transform(reviews)

* Para reducir la dimensionalidad de los vectores, y posteriormente poder graficar los resultados en un minifold 2D, se utilizó Principal Component Analysis (PCA).

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

dataset = pd.read_csv('beer_reviews.csv')
...
reviews = PCA(n_components = 2).fit_transform(reviews)

* Se tomaron los campos **brewery_id** y **beer_beerid** como las formas numéricas de las etiquetas **brewery_name** y **beer_type**. De esta forma, **brewery_id** es equivalente a **brewery_name**, y **beer_beerid** es equivalente a **beer_type**. 

* Todos los análisis que se muestran a continuación se realizaron entrenando los algoritmos de clustering con las siguientes 6 variables enteras del dataset: review_overall, review_aroma, review_appearance, review_palate, review_taste, beer_abv. Las demás variables no numéricas fueron descartadas.

In [ ]:
reviews = pd.DataFrame({'review_overall':dataset['review_overall'], 'review_aroma':dataset['review_aroma'], 'review_appearance':dataset['review_appearance'], 'review_palate':dataset['review_palate'], 'review_taste':dataset['review_taste'],'beer_abv':dataset['beer_abv']})

* Dado que los a los algortimos no se les pasa como parte del dataset las etiquetas que se quieren evaluar, éste proceso de clustering es no supervisado. Es por esto que posterior a la ejecución de los algoritmos de clustering se evalúa la calidad de los clusters encontrados utilizando la función Normalized Mutual Information (NMI), a la cual se le pasa como parámetro las etiquetas reales y las etiquetas obtenidas por los algoritmos, retornando un valor entre 0 y 1 que determina la calidad de los clusters.

In [ ]:
from sklearn.metrics import normalized_mutual_info_score as NMI
...
labels = list(clustering_algorithm.labels_)
NMI(real_labels, labels)

* En este informe se mostrarán sólo los gráficos resultantes del análisis del dataset con cada algoritmo. El código fuente está disponible en https://github.com/matipacheco/Tarea1-MD.

### K-Means

Los parámetros de K-Means utilizados para realizar el análisis fueron los siguientes:

* **init     = 'k-means++'**. Para una convergencia más rápida.
* **n_init   = 10**. Se comprobó que incrementar la cantidad de veces que se ejecuta el algoritmo no mejora significativamente el resultado, sólo se produce un pequeño cambio en la posición de los centroides.
* **max_iter = 300 (default)**. Se comprobó que incrementar la cantidad máxima de iteraciones del algoritmo no mejora el resultado.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

Es importante destacar que se puede determinar la cantidad óptima de clusters verificando el valor obtenido por la evaluación NMI. Índices más grandes indican mejores valores de k. 

Dicho lo anterior, se ejecutó el algoritmo K-Means múltiples veces, probando con distintos valores para el parámetro k, con valores desde k = 2 hasta k = 100 clusters. Para la etiqueta **beer_beerid** con **k = 98** se obtiene el mejor índice NMI, NMI = 0.3003. Del mismo modo, para la etiqueta **brewery_id** también se tiene una cantidad óptima de clusters **k = 98**, con un índice NMI =  0.1299.

A continuación se muestra la imagen que resulta de la ejecución K-Means con 98 clusters. 

<center><img src="PlotsKMeans/kmeans.png"></center>
<center><small>Figura 1: Clustering utilizando K-Means.</small></center>


### Minibatch K-Means

Los parámetros de Minibatch K-Means utilizados para realizar el análisis fueron los siguientes:

* **n_init   = 100**. Se comprobó que incrementar la cantidad de veces que se ejecuta el algoritmo produce mejoras en el posicionamiento de los centroides.
* **batch_size = 1000**. Se comprobó que incrementar el tamaño de los batches, mejora el posicionamiento de los centroides de los clusters, pero para valores mayores a 1000 este cambio es muy poco significativo.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

Al igual que para K-Means, se ejecutó Minibatch K-Means con k = 2 hasta k = 100. En este caso, para la etiqueta **beer_beerid** se tiene una cantidad óptima de clusters **k = 100**, con un índice NMI = 0.2998. Mientras que para la etiqueta **brewery_id** se tiene un índice NMI máximo igual a 0.1299, utilizando la misma cantidad de clusters.

A continuación se muestra la imagen que resulta de la ejecución Minibatch K-Means con 100 clusters. 

<center><img src="PlotsMinibatchKMeans/minibatch.png"></center>
<center><small>Figura 2: Clustering utilizando Minibatch K-Means.</small></center>


### HAC Complete

En primer lugar, es necesario aclarar que para realizar los análisis con el algoritmo HAC, se utilizó un subconjunto de 20.000 elementos del dataset, a diferencia de los dos algoritmos anteriores en los que se analizó el dataset completo. Esto se debe a que para un número mayor de elementos la ejecución del algoritmo consume muchísimos recursos del computador, y por falta de memoria y de componentes hardware más poderosas, el algoritmo siempre terminaba cayéndose.

Los parámetros de HAC utilizados para realizar el análisis fueron los siguientes:

* **linkage   = 'complete'**. Por restricción de la tarea.
* **affinity = 'euclidean'**. Se comprobó que sólo con la métrica cosine se obtenían clusters distintos. Las métricas euclidean, manhattan, l1 y l2 formaban los mismos clusters.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

Al igual que los dos algoritmos anteriores, se ejecutó HAC pasándole como parámetro valores de k = 2 hasta k = 100. En este caso, para la etiqueta **beer_beerid** se tiene una cantidad óptima de clusters **k = 100**, con un índice NMI = 0.4227. Mientras que para la etiqueta **brewery_id** se tiene un índice NMI máximo igual a 0.2332, para **k = 98** clusters. Se puede apreciar claramente que este algoritmo retorna valores de NMI mucho mejores que K-Means y Minibatch K-Means, pero no se puede asegurar que efectivamente HAC sea mejor que los otros dos, dado que el algoritmo se probó con un subset correspondiente al 0.01% del total de datos.

Sin embargo, hay que destacar que para este algoritmo, y a diferencia de los dos anteriores, para cada etiqueta se obtuvieron valores distintos de k. Dicho eso, a continuación se muestra la imagen que resulta de la ejecución de  HAC con 98 y con 100 clusters.

<center><img src="PlotsHAC/hac98.png"></center>
<center><small>Figura 3: Clustering utilizando HAC con 98 clusters.</small></center>

<center><img src="PlotsHAC/hac100.png"></center>
<center><small>Figura 4: Clustering utilizando HAC con 100 clusters.</small></center>


### Ward

Antes de comenzar el analisis, es necesario aclarar que el algoritmo Ward solo se pudo utilizar un subconjunto de 10.000 elementos del dataset. Esto se debe a que para un número mayor de elementos la ejecución del algoritmo consume muchísimos recursos del computador, y por falta de hardware y memoria el algoritmo siempre terminaba cayéndose (al igual que HAC). Por lo que para obtener resultados, se utiliza una muestra poco significativa (10.000 de 1.590.892 datos).

Se ejecuto el algoritmo para k = [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 170, 200, 220, 300, 400, 500, 1000, 5000]. Comparando los resultados del NMI, se nota que el valor se va acercando a 1 (mas rapido para la etiqueta **beer_beerid** que **brewery_id**), por lo que da a entender que Ward se sobreajusta a clusterizar mejor cuando k se acerca al numero de los datos (en este caso 10.000 datos), por lo que NMI no es un buen indicador para encontrar el parametro optimo de clusters con Ward. Dado esto, se mostrara el resultado con 220 clusters elegido arbitrariamente:

<center><img src="PlotsWard/ward_220.png"></center>
<center><small>Figura 5: Clustering utilizando Ward con 220 clusters.</small></center>

Para este caso, para la etiqueta **beer_beerid** se tiene un índice NMI = 0.5026. Mientras que para la etiqueta **brewery_id** se tiene un índice NMI = 0.2768. Con estos resultados se puede concluir que la etiqueta de **Tipo de Cerveza es mas adecuado con este algoritmo**.


### DBScan


Para DBScan, solo se logro utilizar un subconjunto de 250.000 elementos del dataset por problemas de memoria y hardware similares que cuando se utilizo HAC y Ward.

Luego de probar muchos parametros, se encontraron que los mas optimos para realizar el analisis fueron los siguientes:

* **eps   = 0.065**. Para tener un radio maximo bueno y facilitar la generacion de core points.
* **min_samples = 30**. Se comprobo experimentando que este es un valor estable para alcanzar un buen numero de clusters.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos. A continuacion se mostraran los graficos haciendo diferencia por color de los clusters y por tamaño (circulos mas grandes son *Core Points* y los puntos negros son *Noisy Points*). 

El resultado con estos parametros fue el siguiente:

<center><img src="PlotsDBScan/dbscan_135.png"></center>
<center><small>Figura 6: Clustering utilizando DBScan con min_samples = 30 y eps   = 0.065.</small></center>

Como puede notarse, hay bastante Noisy Points (puntos negros), lo que se considera como los datos que no quedaron en ninguno de los 135 clusters. Con esto, para la etiqueta **beer_beerid** se tiene un índice NMI = 0.1684. Mientras que para la etiqueta **brewery_id** se tiene un índice NMI = 0.098. Con estos resultados se tiene que la etiqueta de **Tipo de Cerveza es más adecuado con este algoritmo** aunque los valores de los indicadores sean bajos.


## Conclusiones

A partir de los datos analizados, se deben responder las siguientes preguntas: ¿Qué atributo, Nombre de cervecería o Tipo de cerveza, describe mejor a los cluster como etiquetas de clase?. ¿Hay mejores marcas que otras en
relación a las evaluaciones obtenidas? ¿Se puede definir algún criterio para determinar el mejor tipo de cerveza? 

Despues del trabajo realizado con los 5 algoritmos de clustering, se tiene que para todos el indice NMI muestra que Tipo de Cerveza es una mejor etiqueta para describir a los clusters. Resumiendo los resultados los NMI para la mejor etiqueta fueron de 0.3003, 0.2998, 0.4227, 0.5026 y 0.1684 respectivamente. Sin mebargo, éstos valores no son comparables entre sí ya que, por una parte, los 3 últimos algoritmos no utilizaron el dataset completo para realizar el análisis, y por otra parte, los valores de estos índices fueron obtenidos con distintos valores de k para cada algoritmo.

Pero obviando lo que se señaló recién, y basandose sólo en los valores obtenidos, se puede concluir que Ward fue el algoritmo que mejor realizó la clusterización, obteniendo un índice NMI = 0.5026. Pero es importante señalar que este algoritmo es el que consume más recursos, ya que solo se pudo ejecutar pasándole un subset de 10.000 registros de un total de 1.590.892.

Para terminar, se puede decir que el mejor criterio para elegir el mejor Tipo de Cerveza, sería seleccionar aquellos puntos que en los gráficos están más cerca de la esquina superior derecha, ya que tendran los mejores puntajes de review (aunque estén normalizados). Y un mejor puntaje de review implica que la cerveza es mejor.